In [1]:
import os
import pandapower as pp
import pandapower.networks as pn
import networkx as nx
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from scipy.stats import mode
import osmnx as ox

# Import our custom Bayesian model classes

from bayesgrid import BayesianPowerModel, BayesianFrequencyModel, BayesianDurationModel, BayesianImpedanceModel

from bayesgrid import create_osm_pandapower_network,save_bus_metric_samples,save_power_phase_samples,save_impedance_samples


# Suppress warnings for a cleaner tutorial
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

print("Libraries imported successfully.")

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


Helper functions for saving are defined.
Libraries imported successfully.



# Tutorial 2: Generating a Synthetic Grid from OpenStreetMap

This tutorial demonstrates the full power of `bayesgrid`: creating a complete, synthetic distribution network from scratch using real-world data from OpenStreetMap (OSM).

While Tutorial 1 showed how to use an *existing* `pandapower` test case, this guide will show you how to generate your own network topology based on any location, such as a city, a specific address, or a set of coordinates.

We will cover the complete workflow:
1.  **Define helper functions** to query OSM and process the graph data.
2.  **Create a `pandapower` network** from an OSM road network, including radializing the graph and setting a substation.
3.  **Load the pre-trained `bayesgrid` models.**
4.  **Calculate the hop distances** for the new OSM-based network.
5.  **Generate a full set of synthetic data** (Power, Phase, Reliability, Impedance) for this new network.
6.  **Save all samples** to `.csv` files for analysis or export.




# Step 1: Creating a Pandapower network from OpenStreetMap

Now, we can use our helper functions to easily create a network. We will show 4 different ways to query OSM, but we will only use the first one (by city) for the rest of the tutorial.



## Example 1: This is the simplest way. Just provide a city and country.

In [ ]:
# Our main example
query_city = "Guaraçai, Brazil"
net_city, tree_graph_city = create_osm_pandapower_network(query_city, query_type='city')



print(f"\nCreated network for {query_city}.")
print(net_city)

## Example 2: By Point + Radius
Useful for modeling a specific neighborhood.

In [ ]:
# A point in Santo Amaro, São Paulo
query_point = (-23.649, -46.702) 

# We can specify a different line type for this network
net_point, tree_graph_point = create_osm_pandapower_network(
    query_point, 
    query_type='point', 
    dist=1500, # 1500-meter radius
    line_std_type="48-AL1/8-ST1A 20.0" # Example of a different standard
)
print(net_point)

## Example 3: By Bounding Box
Useful if you have exact map coordinates.

In [ ]:
# A small bounding box in New York City
query_bbox = {
    'north': 40.715, 'south': 40.710,
    'east': -74.005, 'west': -74.010
}
net_bbox, tree_graph_bbox = create_osm_pandapower_network(query_bbox, query_type='bbox')
print(net_bbox)

## Example 4: By Address
Combines geocoding (finding coordinates) with the point query.

In [ ]:
query_address = "São Carlos City Center, Brazil"
net_address, tree_graph_address = create_osm_pandapower_network(
    query_address, 
    query_type='address', 
    dist=100 # 100-meter radius around the address
)
print(net_address)

# Step 2: Visualize the Radial Network
We can visualize any of the networks we just created. For this tutorial we will visualize the point + radius one (example 2)

In [ ]:
net = net_point
tree_graph = tree_graph_point

# Keep these for the next steps
source_bus = net.ext_grid.bus.iloc[0] # This will be 0
graph_from_net = pn.create_nxgraph(net)



substation_node = net.bus.name.iloc[source_bus] # Get original osmnx node ID

# --- 1. Prepare Node Colors and Sizes ---
node_colors = []
node_sizes = []
for node in tree_graph.nodes():
    if str(node) == substation_node:
        node_colors.append('red')   # Highlight color for the substation
        node_sizes.append(100)      # Make it larger
    else:
        node_colors.append('gray')  # Color for other nodes
        node_sizes.append(5)        # Make other nodes small

# --- 2. Create and Customize the Plot ---
fig, ax = ox.plot_graph(
    tree_graph,
    node_color=node_colors,
    node_size=node_sizes,
    node_zorder=2,
    edge_color='black',
    edge_linewidth=1.0,
    edge_alpha=0.8,
    bgcolor='white',
    show=False,
    close=False
)


# Step 3: setup and initialize BHM

* Please be aware: in the background, the traces from the bayesian models are being loaded. This may take some time. 

* The traces being loaded represents previous knowledge learned from a public database. It is possible to create a bayesian model from your own data. To learn how to do it, see notebook tutorial 3: Learning the bhm from new data

* You can define the total demand for the power model, if you want. This will be the total active power (in kW) to be splitted across the buses.

In [ ]:
# --- Set global generation parameters ---
RANDOM_SEED = 42
N_SAMPLES_TO_GENERATE = 100 # We will generate 100 synthetic network samples
OUTPUT_FOLDER = 'new_osm_synthetic_net' # Folder to save our results

# 1. Power & Phase Model
# We set total_demand=None to not rescale the output
bhm = BayesianPowerModel(total_demand=1e3) # 1 GW of total demand

# 2. Frequency Model
bfm = BayesianFrequencyModel()

# 3. Duration Model
bdm = BayesianDurationModel()

# 4. Impedance Model (R and X)
bim = BayesianImpedanceModel()

# Step 4: Calculate Hop & Electrical Distances


We use the net and graph_from_net objects (which we created in Step 2 from OSM data) to calculate the hop distance zones.

In [ ]:
print("Calculating hop distances for all buses in the OSM network...")

# --- 1. Calculate Hop Distance for all buses ---
hop_distances_new_net = {}
for bus_idx in net.bus.index:
    try:
        dist = nx.shortest_path_length(graph_from_net, source=source_bus, target=bus_idx)
        hop_distances_new_net[bus_idx] = dist
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        hop_distances_new_net[bus_idx] = np.nan

hop_series = pd.Series(hop_distances_new_net, index=net.bus.index)
max_dist = hop_series.max()
hop_series = hop_series.fillna(max_dist)
print(f"Max hop distance found: {max_dist}")

# --- 2. Discretize Bus Hop Distances for RELIABILITY (30 Bins) ---
N_BINS_RELIABILITY = 30
hop_zone_idx_reliability = pd.cut(
    hop_series, bins=N_BINS_RELIABILITY, labels=False, include_lowest=True
).values
print(f"Created {len(hop_zone_idx_reliability)} bus zone indices for reliability (30 bins).")

# --- 3. Discretize Bus Hop Distances for POWER (10 Bins) ---
N_BINS_POWER_IMPD = 10
hop_zone_series_power = pd.cut(
    hop_series, bins=N_BINS_POWER_IMPD, labels=False, include_lowest=True
)
hop_zone_idx_power = hop_zone_series_power.values
print(f"Created {len(hop_zone_idx_power)} bus zone indices for power (10 bins).")

# --- 4. Discretize LINE Electrical Distances (10 Bins) ---
from_buses = net.line.from_bus
line_elec_dist_idx = hop_zone_series_power.loc[from_buses].values
print(f"Created {len(line_elec_dist_idx)} line zone indices for impedance (10 bins).")

# Step 5: Generate All Synthetic Data
We now feed our new OSM-based zone indices into the models. The models don't care where the zones came from, only that they are provided.



In [ ]:
len(graph_from_net.nodes)

In [ ]:
print("--- Starting Synthetic Data Generation ---")

# --- 1. Generate Power & Phase ---
print("\nGenerating Power and Phase...")
gen_phases, gen_power = bhm.generate_consistent_data(
    new_hop_zone_idx=hop_zone_idx_power,
    graph=graph_from_net,
    source_bus_idx=source_bus,
    scan_draws=N_SAMPLES_TO_GENERATE,
    random_seed=RANDOM_SEED
)
print(f"Power/Phase shape: {gen_power.shape}")

# --- 2. Generate Failure Frequency ---
print("\nGenerating Failure Frequency (CAIFI/FIC)...")
gen_freq_all = bfm.generate_data(
    new_hop_zone_idx=hop_zone_idx_reliability, random_seed=RANDOM_SEED
)
gen_freq = gen_freq_all[:N_SAMPLES_TO_GENERATE, :]
print(f"Frequency shape: {gen_freq.shape}")

# --- 3. Generate Failure Duration ---
print("\nGenerating Failure Duration (CAIDI/DIC)...")
gen_dur_all = bdm.generate_data(
    new_hop_zone_idx=hop_zone_idx_reliability, random_seed=RANDOM_SEED
)
gen_dur = gen_dur_all[:N_SAMPLES_TO_GENERATE, :]
print(f"Duration shape: {gen_dur.shape}")

# --- 4. Generate R1 and X1 Impedance ---
print("\nGenerating Line Impedance (R1/X1)...")
gen_r_all, gen_x_all = bim.generate_data(
    new_elec_dist_idx=line_elec_dist_idx, random_seed=RANDOM_SEED
)
gen_r = gen_r_all[:N_SAMPLES_TO_GENERATE, :]
gen_x = gen_x_all[:N_SAMPLES_TO_GENERATE, :]
print(f"R1 shape: {gen_r.shape}, X1 shape: {gen_x.shape}")

print("\n--- All Generation Complete ---")

# Step 6: Post-Process and Save All Samples
Finally, we save all N_SAMPLES_TO_GENERATE into our new output folder, new_osm_synthetic_net.

In [ ]:
gen_power = gen_power[:N_SAMPLES_TO_GENERATE, :, :]

gen_phases = gen_phases[:N_SAMPLES_TO_GENERATE, :]

gen_freq = gen_freq[:N_SAMPLES_TO_GENERATE, :]

gen_dur = gen_dur[:N_SAMPLES_TO_GENERATE, :]

gen_r = gen_r[:N_SAMPLES_TO_GENERATE, :]

gen_x = gen_x[:N_SAMPLES_TO_GENERATE, :]

In [ ]:
print(f"Processing all {N_SAMPLES_TO_GENERATE} samples and saving to folder: '{OUTPUT_FOLDER}'")
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# 1. Save Power and Phase
save_power_phase_samples(
    gen_phases=gen_phases,
    gen_power=gen_power,
    bus_index=net.bus.index,
    phase_map=bhm.get_phase_map(),
    n_samples=N_SAMPLES_TO_GENERATE,
    output_path=os.path.join(OUTPUT_FOLDER, 'bus_power_and_phase_SAMPLES.csv')
)

# 2. Save Frequency
save_bus_metric_samples(
    gen_data=gen_freq,
    col_name='CAIFI_FIC',
    bus_index=net.bus.index,
    n_samples=N_SAMPLES_TO_GENERATE,
    output_path=os.path.join(OUTPUT_FOLDER, 'bus_frequency_SAMPLES.csv')
)

# 3. Save Duration
save_bus_metric_samples(
    gen_data=gen_dur,
    col_name='CAIDI_DIC',
    bus_index=net.bus.index,
    n_samples=N_SAMPLES_TO_GENERATE,
    output_path=os.path.join(OUTPUT_FOLDER, 'bus_duration_SAMPLES.csv')
)

# 4. Save Impedance
save_impedance_samples(
    gen_r=gen_r,
    gen_x=gen_x,
    line_index=net.line.index,
    n_samples=N_SAMPLES_TO_GENERATE,
    output_path=os.path.join(OUTPUT_FOLDER, 'line_impedance_SAMPLES.csv')
)

print("\n--- All Synthetic Data Samples Saved Successfully! ---")

In [ ]:
pp.to_pickle(net,os.path.join(OUTPUT_FOLDER,'pandapower_network.p'))

# Tutorial Complete
You have successfully started from a simple OpenStreetMap query (address, bounding box, point location, city name); downloaded real-world map data;  converted it into a radial pandapower network, and generated a full set of N_SAMPLES_TO_GENERATE synthetic parameters for it.